ASAP CRN Metadata compilation

# Team Hardy. ASAP CRN Metadata scrubbing

confirm with Team Hardy bulkRNAseq metadata

16 Oct 2024
Andy Henrie





In [1]:
import pandas as pd
from pathlib import Path
import os, sys

sys.path.append(os.path.abspath((os.path.join(os.getcwd(), 'src/crn_utils'))))

from util import read_CDE, NULL, prep_table, read_meta_table, create_metadata_package
from validate import validate_table, ReportCollector
from update_schema import v1_to_v2, v2_to_v3_PMDBS, intervention_typer
from checksums import extract_md5_from_details2, get_md5_hashes 
from bucket_util import authenticate_with_service_account, gsutil_ls, gsutil_cp, gsutil_mv 

%load_ext autoreload
%autoreload 2

root_path = Path.home() / ("Projects/ASAP/data/teams")


Streamlit NOT successfully imported


## CDEs
load the relavent CDEs

In [2]:
schema_version = "v1"
schema_path = Path.home() / "Projects/ASAP/crn-utils/resource/CDE"
CDEv1 = read_CDE(schema_version, local_path=schema_path)
schema_version = "v2.1"
CDEv2 = read_CDE(schema_version, local_path=schema_path)
schema_version = "v3.0"
CDEv3 = read_CDE(schema_version, local_path=schema_path)

metadata_version: ASAP_CDE_v1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v1
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v1.csv
read local file
metadata_version: ASAP_CDE_v2.1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v2.1
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v2.1.csv
read local file
metadata_version: ASAP_CDE_v3.0
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v3.0
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v3.0.csv
read local file


## Load original tables 
These were submitted v3.0 (actually  v3.0.0 beta)


Team Hardy bulk-rnaseq


In [3]:
## convert 
team = "hardy"
dataset_name = "bulk-rnaseq"

metadata_path = root_path / f"{team}/{dataset_name}/metadata/"
v3_path = metadata_path / "v3"
og_path = metadata_path / "og"

v3_meta_tables = ['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DATA', 'CLINPATH', 'PMDBS', 'CONDITION', 'ASSAY_RNAseq']
og_meta_tables = ['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DATA', 'CLINPATH', 'PMDBS', 'CONDITION', 'ASSAY_bulkRNAseq']

in_tables = [table_name for table_name in og_meta_tables if f"{table_name}.csv" in os.listdir(og_path)]


In [4]:
og_tables = {}
for table_name in in_tables:
    df = read_meta_table(f"{og_path}/{table_name}.csv")
    og_tables[table_name] = df
    

In [5]:
og_tables.keys()

dict_keys(['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DATA', 'CLINPATH', 'PMDBS', 'CONDITION', 'ASSAY_bulkRNAseq'])

In [6]:
og_tables['STUDY']

,ASAP_team_name,ASAP_lab_name,project_name,team_dataset_id,project_dataset,project_description,PI_full_name,PI_email,contributor_names,submitter_name,...,publication_PMID,number_samples,sample_types,types_of_samples,DUA_version,PI_ORCHID,PI_google_scholar_id,preprocessing_references,metadata_version_date,alternate_dataset_id
0,TEAM-HARDY,Ryten Lab,Understanding mechanisms of Parkinson's diseas...,Hardy_bulkrnaseq,Hardy_bulkrnaseq_pm_hs,Genetic analysis has identified many risk gene...,Mina Ryten,mina.ryten@ucl.ac.uk,"Aine Fairbrother-Browne, Jonathan Brenton, Mel...",Jonathan Brenton,...,NA,288,"Anterior Cingulate Cortex (ACG), Inferior Pari...",Late stage (Braak 6) PD and control post-morte...,0000-0001-9520-6957,https://scholar.google.co.uk/citations?user=lt...,NA,"Version 3.0 - beta, 08/2024",NA,NA


In [7]:
# rationalize the team_dataset_id
og_tables['STUDY']['team_dataset_id'] = dataset_name.replace("-", "_").replace(" ", "_")

# fix the PI_ORCID and PI_google_scholar_id
og_tables['STUDY']['PI_google_scholar_id'] = og_tables['STUDY']['PI_ORCHID']
og_tables['STUDY']['PI_ORCHID'] = "0000-0001-9520-6957"
# rename PI_ORCHID to PI_ORCID
og_tables['STUDY'].rename(columns={"PI_ORCHID": "PI_ORCID"}, inplace=True)

In [8]:
CDE = CDEv3
dfs = {}
for table,df in og_tables.items():
    if table == "ASSAY_bulkRNAseq":
        table = "ASSAY_RNAseq"

    schema = CDE[CDE['Table'] == table]

    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    report.print_log()
    dfs[table] = full_table

recoding number_samples as int
🚨⚠️❗ **Missing Required Fields in STUDY: metadata_tables**
🚨⚠️❗ **5 Fields with empty (NULL) values:**

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_DOI: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- metadata_version_date: 1/1 empty rows (OPTIONAL)

	- alternate_dataset_id: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *PROTOCOL* table.
🚨⚠️❗ **3 Fields with empty (NULL) values:**

	- cell_extraction_summary: 1/1 empty rows (REQUIRED)

	- protocols_io_DOI: 1/1 empty rows (REQUIRED)

	- other_reference: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *SUBJECT* table.
No empty entries (NULL) found .
No invalid entries found in Enum fields.
🚨⚠️❗ **Extra field in SUBJECT: organism**
🚨⚠️❗ **Extra field in SUBJECT: ethnicity**
🚨⚠️❗ **Extra field in SUBJECT: GP2_id**
🚨⚠️❗ **Extra field in SUBJECT: ag

In [9]:
STUDY = dfs['STUDY']
PROTOCOL = dfs['PROTOCOL']
SUBJECT = dfs['SUBJECT']
SAMPLE = dfs['SAMPLE']
DATA = dfs['DATA']
CLINPATH = dfs['CLINPATH']
PMDBS = dfs['PMDBS']
ASSAY_RNAseq = dfs['ASSAY_RNAseq']

In [10]:
STUDY

,ASAP_team_name,ASAP_lab_name,project_name,team_dataset_id,project_dataset,project_description,PI_full_name,PI_email,contributor_names,submitter_name,...,number_samples,sample_types,types_of_samples,DUA_version,PI_ORCID,PI_google_scholar_id,preprocessing_references,metadata_version_date,alternate_dataset_id,metadata_tables
0,TEAM-HARDY,Ryten Lab,Understanding mechanisms of Parkinson's diseas...,bulk_rnaseq,Hardy_bulkrnaseq_pm_hs,Genetic analysis has identified many risk gene...,Mina Ryten,mina.ryten@ucl.ac.uk,"Aine Fairbrother-Browne, Jonathan Brenton, Mel...",Jonathan Brenton,...,288,"Anterior Cingulate Cortex (ACG), Inferior Pari...",Late stage (Braak 6) PD and control post-morte...,0000-0001-9520-6957,0000-0001-9520-6957,https://scholar.google.co.uk/citations?user=lt...,"Version 3.0 - beta, 08/2024",NA,NA,NA


In [11]:
STUDY['metadata_tables'] = f"{v3_meta_tables}"

In [12]:
metadata_version = "v3.0"
METADATA_VERSION_DATE = f"{metadata_version}_{pd.Timestamp.now().strftime('%Y%m%d')}"
STUDY['metadata_version_date'] = METADATA_VERSION_DATE


In [ ]:
# 🚨⚠️❗ the source_subject_id for this bulk set overlap with the sn Hardy dataset.  However
# that dataset specifies different internal subject_ids (e.g. babom, borah, etc.)  Likewise, the sample_id are 
# inconsistent between the two datasets.  Fortunately, the source_subject_id saves us.  ASAP ID generation
# works correctly, and sample / subject IDs are correct at the ASAP_ID level.


# fixe the subject_id by adding Hardy and removing the / from the source_subject_id
SUBJECT['subject_id'] = "Hardy_" + SUBJECT['source_subject_id'].str.replace("/", "_")

# the subject_id is actually a sample_id, so we will also need to remove duplicates.
SUBJECT = SUBJECT.drop_duplicates(subset=['subject_id'])



# # fix the 'subject_id' by making a mapper from 'source_subject_id' to 'subject_id' from CLINPATH
# subject_mapper = dict(zip(CLINPATH['source_subject_id'], CLINPATH['subject_id']))
# SUBJECT['subject_id'] = SUBJECT['source_subject_id'].map(subject_mapper)


In [14]:
table = "SUBJECT"
schema = CDE[CDE['Table'] == table]

report = ReportCollector(destination="NA")
full_table, report = validate_table(dfs[table].copy(), table, schema, report)
report.print_log()


All required fields are present in *SUBJECT* table.
No empty entries (NULL) found .
No invalid entries found in Enum fields.
🚨⚠️❗ **Extra field in SUBJECT: organism**
🚨⚠️❗ **Extra field in SUBJECT: ethnicity**
🚨⚠️❗ **Extra field in SUBJECT: GP2_id**
🚨⚠️❗ **Extra field in SUBJECT: age_at_diagnosis**



In [15]:
table = "SAMPLE"
schema = CDE[CDE['Table'] == table]

report = ReportCollector(destination="NA")
full_table, report = validate_table(dfs[table].copy(), table, schema, report)
report.print_log()

recoding replicate_count as int
recoding repeated_sample as int
All required fields are present in *SAMPLE* table.
🚨⚠️❗ **17 Fields with empty (NULL) values:**

	- organism: 288/288 empty rows (REQUIRED)

	- assay_type: 288/288 empty rows (REQUIRED)

	- condition_id: 288/288 empty rows (REQUIRED)

	- organism_ontology_term_id: 288/288 empty rows (REQUIRED)

	- time: 288/288 empty rows (OPTIONAL)

	- development_stage_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- sex_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- self_reported_ethnicity_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- disease_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- tissue_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- assay_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- donor_id: 288/288 empty rows (OPTIONAL)

	- pm_PH: 288/288 empty rows (OPTIONAL)

	- cell_type_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- source_RIN: 288/288 empty rows (OPTIONAL)

	- DV200: 288/288 empty rows

In [16]:
# get organism from SUBJECT
SAMPLE['organism'] = SUBJECT['organism'].iloc[0]
SAMPLE['condition_id'] = SAMPLE['condition'].str.replace(", ", "-").str.replace(" ", "_")
SAMPLE['condition_id'].unique()

# fixe the subject_id by adding Hardy and removing the / from the source_sample_id
SAMPLE['subject_id'] = "Hardy_" + SAMPLE['source_sample_id'].str.replace("/", "_")
SAMPLE['source_sample_id'] = NULL

In [17]:
SUBJECT[['subject_id','source_subject_id','GP2_id','organism','ethnicity','GP2_id','age_at_diagnosis']].head()

,subject_id,source_subject_id,GP2_id,organism,ethnicity,GP2_id,age_at_diagnosis
0,Hardy_P27_11,P27/11,MDGAP-QSBB_000088_s1,Human,NA,MDGAP-QSBB_000088_s1,69
4,Hardy_P8_18,P8/18,MDGAP-QSBB_000583_s1,Human,NA,MDGAP-QSBB_000583_s1,61
8,Hardy_P41_09,P41/09,MDGAP-QSBB_000406_s1,Human,NA,MDGAP-QSBB_000406_s1,NA
12,Hardy_P26_11,P26/11,MDGAP-QSBB_000325_s1,Human,NA,MDGAP-QSBB_000325_s1,NA
16,Hardy_P95_10,P95/10,MDGAP-QSBB_000046_s1,Human,NA,MDGAP-QSBB_000046_s1,71


In [18]:
table = "CLINPATH"
schema = CDE[CDE['Table'] == table]

report = ReportCollector(destination="NA")
full_table, report = validate_table(dfs[table].copy(), table, schema, report)
report.print_log()

recoding age_at_onset as int
recoding age_at_diagnosis as int
recoding first_motor_symptom as int
All required fields are present in *CLINPATH* table.
🚨⚠️❗ **38 Fields with empty (NULL) values:**

	- duration_pmi: 4/288 empty rows (REQUIRED)

	- AMPPD_id: 288/288 empty rows (OPTIONAL)

	- GP2_id: 288/288 empty rows (OPTIONAL)

	- ethnicity: 288/288 empty rows (OPTIONAL)

	- family_history: 288/288 empty rows (OPTIONAL)

	- last_diagnosis: 288/288 empty rows (OPTIONAL)

	- age_at_onset: 288/288 empty rows (OPTIONAL)

	- age_at_diagnosis: 288/288 empty rows (OPTIONAL)

	- first_motor_symptom: 288/288 empty rows (OPTIONAL)

	- hx_dementia_mci: 288/288 empty rows (OPTIONAL)

	- hx_melanoma: 288/288 empty rows (OPTIONAL)

	- education_level: 288/288 empty rows (OPTIONAL)

	- smoking_status: 288/288 empty rows (OPTIONAL)

	- smoking_years: 288/288 empty rows (OPTIONAL)

	- APOE_e4_status: 288/288 empty rows (OPTIONAL)

	- cognitive_status: 288/288 empty rows (OPTIONAL)

	- time_from_baseline

In [19]:
# copy 'GP2_id', 'age_at_diagnosis' from SUBJECT
gp2_mapper = dict(zip(SUBJECT['subject_id'], SUBJECT['GP2_id']))
age_mapper = dict(zip(SUBJECT['subject_id'], SUBJECT['age_at_diagnosis']))


# fixe the subject_id by adding Hardy and removing the / from the source_subject_id
CLINPATH['subject_id'] = "Hardy_" + CLINPATH['source_subject_id'].str.replace("/", "_")

# the subject_id is actually a sample_id, so we will also need to remove duplicates.
CLINPATH = CLINPATH.drop_duplicates(subset=['subject_id'])

CLINPATH['GP2_id'] = CLINPATH['subject_id'].map(gp2_mapper)
CLINPATH['age_at_diagnosis'] = CLINPATH['subject_id'].map(age_mapper)
CLINPATH[['subject_id','source_subject_id','GP2_id','age_at_diagnosis']]

/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_44021/1833959381.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CLINPATH['GP2_id'] = CLINPATH['subject_id'].map(gp2_mapper)
/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_44021/1833959381.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CLINPATH['age_at_diagnosis'] = CLINPATH['subject_id'].map(age_mapper)


,subject_id,source_subject_id,GP2_id,age_at_diagnosis
0,Hardy_P27_11,P27/11,MDGAP-QSBB_000088_s1,69
4,Hardy_P8_18,P8/18,MDGAP-QSBB_000583_s1,61
8,Hardy_P41_09,P41/09,MDGAP-QSBB_000406_s1,NA
12,Hardy_P26_11,P26/11,MDGAP-QSBB_000325_s1,NA
16,Hardy_P95_10,P95/10,MDGAP-QSBB_000046_s1,71
...,...,...,...,...
268,Hardy_P11_17,P11/17,MDGAP-DEMENTIA_000251_s1,NA
272,Hardy_P62_15,P62/15,MDGAP-QSBB_000517_s1,80
276,Hardy_P68_05,P68/05,MDGAP-QSBB_000014_s1,62
280,Hardy_P52_13,P52/13,MDGAP-QSBB_000081_s1,64


In [20]:
table = "SAMPLE"
schema = CDE[CDE['Table'] == table]

report = ReportCollector(destination="NA")
full_table, report = validate_table(dfs[table].copy(), table, schema, report)
report.print_log()

recoding replicate_count as int
recoding repeated_sample as int
All required fields are present in *SAMPLE* table.
🚨⚠️❗ **16 Fields with empty (NULL) values:**

	- source_sample_id: 288/288 empty rows (REQUIRED)

	- assay_type: 288/288 empty rows (REQUIRED)

	- organism_ontology_term_id: 288/288 empty rows (REQUIRED)

	- time: 288/288 empty rows (OPTIONAL)

	- development_stage_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- sex_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- self_reported_ethnicity_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- disease_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- tissue_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- assay_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- donor_id: 288/288 empty rows (OPTIONAL)

	- pm_PH: 288/288 empty rows (OPTIONAL)

	- cell_type_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- source_RIN: 288/288 empty rows (OPTIONAL)

	- DV200: 288/288 empty rows (OPTIONAL)

	- suspension_type: 288/288

In [21]:
# CONDITION
# construct this table.  needs to be checked by hand
CONDITIONv3 = pd.DataFrame(columns=CDEv3[CDEv3['Table'] == "CONDITION"]['Field'])
CONDITIONv3['condition_id'] = SUBJECT['primary_diagnosis'].unique()
CONDITIONv3['intervention_name'] = "Case-Control"
CONDITIONv3['intervention_id'] = CONDITIONv3['condition_id'].apply(intervention_typer)
CONDITIONv3['condition_id'] = CONDITIONv3['condition_id'].str.lower().str.replace(" ", "_")

CONDITIONv3

Field,condition_id,intervention_name,intervention_id,protocol_id,intervention_aux_table
0,idiopathic_pd,Case-Control,Case,NaN,NaN
1,no_pd_nor_other_neurological_disorder,Case-Control,Control,NaN,NaN
2,other_neurological_disorder,Case-Control,Other,NaN,NaN


In [22]:
DATA[['sample_id','file_name','file_MD5']]['file_MD5'][0]

'c78d76759a539f203eafc53165f14ab2'

In [23]:
# now save the tables
v3_tables = {
    "STUDY": STUDY,
    "PROTOCOL": PROTOCOL,
    "SUBJECT": SUBJECT,
    "SAMPLE": SAMPLE,
    "DATA": DATA,
    "CLINPATH": CLINPATH,
    "PMDBS": PMDBS,
    "CONDITION": CONDITIONv3,
    "ASSAY_RNAseq": ASSAY_RNAseq
}

### save extras as auxillary tables


In [24]:

for table,df in v3_tables.items():
    schema = CDE[CDE['Table'] == table]
    valid_fields = schema['Field'].unique()
    df_out = df[valid_fields]
    aux_fields = set(df.columns) - set(valid_fields)
    if aux_fields:
        df_aux = df[list(aux_fields)]
        df_aux.to_csv(og_path / f"{table}_auxiliary.csv", index=False)
        print(f"Saved {table}_auxiliary.csv")
    df_out.to_csv(v3_path / f"{table}.csv", index=False)

Saved SUBJECT_auxiliary.csv
Saved SAMPLE_auxiliary.csv
Saved ASSAY_RNAseq_auxiliary.csv


### validate v3 tables


In [25]:
CDE = CDEv3
tables = CDE['Table'].unique()
v3_meta_tables = ['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DATA', 'CLINPATH', 'PMDBS', 'CONDITION', 'ASSAY_RNAseq']

v3_tables = {}
for table in v3_meta_tables:
    df = read_meta_table(v3_path / f"{table}.csv")
    schema = CDE[CDE['Table'] == table]

    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    report.print_log()
    v3_tables[table] = full_table

recoding number_samples as int
All required fields are present in *STUDY* table.
🚨⚠️❗ **4 Fields with empty (NULL) values:**

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_DOI: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- alternate_dataset_id: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *PROTOCOL* table.
🚨⚠️❗ **3 Fields with empty (NULL) values:**

	- cell_extraction_summary: 1/1 empty rows (REQUIRED)

	- protocols_io_DOI: 1/1 empty rows (REQUIRED)

	- other_reference: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *SUBJECT* table.
No empty entries (NULL) found .
No invalid entries found in Enum fields.

recoding replicate_count as int
recoding repeated_sample as int
All required fields are present in *SAMPLE* table.
🚨⚠️❗ **16 Fields with empty (NULL) values:**

	- source_sample_id: 288/288 empty rows (REQUIRED)

	- ass

-------------------------
## check md5s



In [26]:
print(team)

source = "pmdbs"

bucket = f"asap-raw-team-{team}-{source}-{dataset_name}"

key_file_path = Path.home() / f"Projects/ASAP/{team}-credentials.json"

res = authenticate_with_service_account(key_file_path)
print(res)

# make sure to get ALL the fastq files in the bucket
prefix = "**/*.gz"
bucket_files_md5 = get_md5_hashes( bucket, prefix)

hardy
CompletedProcess(args='gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/hardy-credentials.json', returncode=0, stdout='', stderr='Activated service account credentials for: [raw-admin-hardy@dnastack-asap-parkinsons.iam.gserviceaccount.com]\n')
gsutil -u dnastack-asap-parkinsons hash -h gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/**/*.gz


In [27]:
# def check_md5_sums()


checksum = v3_tables['DATA'][['file_name','file_MD5']]
checksum['check2'] = checksum['file_name'].map(bucket_files_md5)
checksum['check1'] = checksum['file_MD5']
checksum[checksum.check1 != checksum.check2].file_name.to_list()
#empty means success!!

/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_44021/1352623229.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check2'] = checksum['file_name'].map(bucket_files_md5)
/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_44021/1352623229.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check1'] = checksum['file_MD5']


[]

_____

## prep metadata in raw data bucket

steps:
- 1. archive whats there.  i.e. move to metadata/upload
- 2. copy metadata/upload to dataset upload (upload subdir)

In [28]:
metadata_subdir = "metadata"
current_files = gsutil_ls(bucket,metadata_subdir)

gsutil -u dnastack-asap-parkinsons ls gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata
gsutil command succeeded: gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/ASSAY_bulkRNAseq.csv
gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/CLINPATH.csv
gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/DATA.csv
gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/PMDBS.csv
gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/PROTOCOL.csv
gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/SAMPLE.csv
gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/STUDY.csv
gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/SUBJECT.csv



In [29]:
metadata_subdir2 = "metadata/upload"
bucket = current_files[0].split("/")[2]

for file in current_files:
    if file == "":
        continue
    file_nm = Path(file).name
    is_dir = not file_nm.__contains__(".")

    source = f"gs://{bucket}/{metadata_subdir}/{file_nm}"

    destination = f"gs://{bucket}/{metadata_subdir2}/{file_nm}"
    gsutil_mv(source, destination, is_dir)

gsutil -u dnastack-asap-parkinsons mv gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/ASSAY_bulkRNAseq.csv gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/upload/ASSAY_bulkRNAseq.csv
gsutil command succeeded: 
gsutil -u dnastack-asap-parkinsons mv gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/CLINPATH.csv gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/upload/CLINPATH.csv
gsutil command succeeded: 
gsutil -u dnastack-asap-parkinsons mv gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/DATA.csv gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/upload/DATA.csv
gsutil command succeeded: 
gsutil -u dnastack-asap-parkinsons mv gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/PMDBS.csv gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/upload/PMDBS.csv
gsutil command succeeded: 
gsutil -u dnastack-asap-parkinsons mv gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/PROTOCOL.csv gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/upload/PROTOCOL.csv
gsutil command 

In [30]:

metadata_subdir = "metadata/upload"
current_files = gsutil_ls(bucket,metadata_subdir)

gsutil -u dnastack-asap-parkinsons ls gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/upload
gsutil command succeeded: gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/upload/ASSAY_bulkRNAseq.csv
gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/upload/CLINPATH.csv
gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/upload/DATA.csv
gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/upload/PMDBS.csv
gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/upload/PROTOCOL.csv
gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/upload/SAMPLE.csv
gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/upload/STUDY.csv
gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/upload/SUBJECT.csv



Archive the uploaded metadata locally



In [31]:
file_source = f"gs://{bucket}/{metadata_subdir}"
destination = f"{metadata_path}"

In [32]:

is_dir = True
gsutil_cp(file_source, destination, is_dir)

gsutil -u dnastack-asap-parkinsons cp -r gs://asap-raw-team-hardy-pmdbs-bulk-rnaseq/metadata/upload /Users/ergonyc/Projects/ASAP/data/teams/hardy/bulk-rnaseq/metadata
gsutil command succeeded: 


''

--------------------
## Create metadata package

This will copy the final updated to v3.0 metadata to `asap-could-processing-resources`


In [33]:
metadata_source = metadata_path


source = "pmdbs"
archive_root = Path.home() / "Projects/ASAP/asap-crn-metadata/datasets"
dataset_path = archive_root / f"{team}-{source}-{dataset_name}"
# bucket = f"asap-raw-data-team-{team}" # for now old locations
metadata_source, dataset_path

(PosixPath('/Users/ergonyc/Projects/ASAP/data/teams/hardy/bulk-rnaseq/metadata'),
 PosixPath('/Users/ergonyc/Projects/ASAP/asap-crn-metadata/datasets/hardy-pmdbs-bulk-rnaseq'))

________

In [34]:
fnms = create_metadata_package(metadata_source, dataset_path)


Skipping empty folder /Users/ergonyc/Projects/ASAP/data/teams/hardy/bulk-rnaseq/metadata/v1
Skipping empty folder /Users/ergonyc/Projects/ASAP/data/teams/hardy/bulk-rnaseq/metadata/v2
Copied /Users/ergonyc/Projects/ASAP/data/teams/hardy/bulk-rnaseq/metadata/v3 to /Users/ergonyc/Projects/ASAP/asap-crn-metadata/datasets/hardy-pmdbs-bulk-rnaseq/metadata/v3
Copied /Users/ergonyc/Projects/ASAP/data/teams/hardy/bulk-rnaseq/metadata/og to /Users/ergonyc/Projects/ASAP/asap-crn-metadata/datasets/hardy-pmdbs-bulk-rnaseq/metadata/og
Copied /Users/ergonyc/Projects/ASAP/data/teams/hardy/bulk-rnaseq/metadata/upload to /Users/ergonyc/Projects/ASAP/asap-crn-metadata/datasets/hardy-pmdbs-bulk-rnaseq/metadata/upload


_____

generate ASAP IDs + transfering back to raw data bucket via `asap-crn-metadata` 

_____

## transfer metadata to raw data bucket

steps:
- 1. archive whats there.  i.e. move to metadata/archive
- 2. copy package to metadata/ . i.e. /og/*.csv, /v??/*.csv

_____

## prep metadata in raw data bucket

steps:
- 1. archive whats there.  i.e. move to metadata/upload
- 2. copy metadata/upload to dataset upload (upload subdir)

In [ ]:
fnms = create_metadata_package(metadata_source, dataset_path)


Copied /Users/ergonyc/Projects/ASAP/data/teams/hardy/sn-rnaseq/metadata/v1 to /Users/ergonyc/Projects/ASAP/asap-crn-metadata/datasets/hardy-pmdbs-sn-rnaseq/metadata/v1
Copied /Users/ergonyc/Projects/ASAP/data/teams/hardy/sn-rnaseq/metadata/v2 to /Users/ergonyc/Projects/ASAP/asap-crn-metadata/datasets/hardy-pmdbs-sn-rnaseq/metadata/v2
Copied /Users/ergonyc/Projects/ASAP/data/teams/hardy/sn-rnaseq/metadata/v3 to /Users/ergonyc/Projects/ASAP/asap-crn-metadata/datasets/hardy-pmdbs-sn-rnaseq/metadata/v3
Copied /Users/ergonyc/Projects/ASAP/data/teams/hardy/sn-rnaseq/metadata/og to /Users/ergonyc/Projects/ASAP/asap-crn-metadata/datasets/hardy-pmdbs-sn-rnaseq/metadata/og
Copied /Users/ergonyc/Projects/ASAP/data/teams/hardy/sn-rnaseq/metadata/upload to /Users/ergonyc/Projects/ASAP/asap-crn-metadata/datasets/hardy-pmdbs-sn-rnaseq/metadata/upload


_____

generate ASAP IDs + transfering back to raw data bucket via `asap-crn-metadata` 

_____

## transfer metadata to raw data bucket

steps:
- 1. archive whats there.  i.e. move to metadata/archive
- 2. copy package to metadata/ . i.e. /og/*.csv, /v??/*.csv

______________